In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.metrics import mean_squared_error, r2_score

# Function to train a model for a specific phase
def train_model(phase_name, phase_file):
    # Load the dataset for the given phase
    df = pd.read_csv(phase_file)

    # Data Cleaning: Drop rows with missing values
    df = df.dropna()

    # Define phase-specific features
    phase_features = {
        "phase_1": ['FF', 'APUF_Mean', 'GS_Mean', 'Distance'],
        "phase_2": ['FF', 'N1T_Mean', 'GS_Mean', 'SAT', 'Distance'],
        "phase_3": ['FF', 'N1T_Mean', 'TAT', 'PS_Mean', 'Distance'],
        "phase_4": ['FF', 'N1T_Mean', 'ALTR_Mean', 'GS_Mean', 'Distance'],
        "phase_5": ['FF', 'TAS_Mean', 'MACH_Mean', 'TAT', 'Distance'],
        "phase_6": ['FF', 'GS_Mean', 'TAT', 'PT_Mean', 'Distance'],
        "phase_7": ['FF', 'GS_Mean', 'TRKM_Mean', 'LATP', 'Distance']
    }

    # Select the relevant features and target for the phase
    features = phase_features.get(phase_name)
    if not features:
        print(f"Invalid phase name: {phase_name}")
        return

    target = 'FQTY_Change'  # Assuming the target column is 'FQTY_Change'
    X = df[features]
    y = df[target]

    # Normalize/Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Train a Random Forest Regressor model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Save the model and the scaler to files
    joblib.dump(model, f"model_{phase_name}.joblib")
    joblib.dump(scaler, f"scaler_{phase_name}.joblib")

    # Model Evaluation (optional)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Phase: {phase_name} - MSE: {mse}, R2: {r2}")

# Example: Training models for each phase
if __name__ == "__main__":
    # Specify the phase names and corresponding files
    phase_files = {
        "phase_1": "/content/SDG_9/phase_1.csv",  # Preflight
        "phase_2": "/content/SDG_9/phase_2.csv",  # Taxi
        "phase_3": "/content/SDG_9/phase_3.csv",  # Takeoff
        "phase_4": "/content/SDG_9/phase_4.csv",  # Climb
        "phase_5": "/content/SDG_9/phase_5.csv",  # Cruise
        "phase_6": "/content/SDG_9/phase_6.csv",  # Approach
        "phase_7": "/content/SDG_9/phase_7.csv"   # Rollout
    }

    # Train models for all phases
    for phase_name, file_path in phase_files.items():
        train_model(phase_name, file_path)


Phase: phase_1 - MSE: 170.1315575418994, R2: 0.20577041364390813
Phase: phase_2 - MSE: 58635.99964835165, R2: 0.33898111949319276
Phase: phase_3 - MSE: 20134.635540983603, R2: 0.9341114965252224
Phase: phase_4 - MSE: 31134.771342222226, R2: 0.9353926450584398
Phase: phase_5 - MSE: 201796.14346666666, R2: 0.9603523778558437
Phase: phase_6 - MSE: 35499.60558222222, R2: 0.7450494940051786
Phase: phase_7 - MSE: 4193.78556424581, R2: 0.1579695196096047


In [61]:
import numpy as np
import joblib

# Function to predict fuel quantity change based on phase
def predict_fuel_quantity(phase_name, input_data):
    # Load the trained model and scaler for the given phase
    model = joblib.load(f"model_{phase_name}.joblib")
    scaler = joblib.load(f"scaler_{phase_name}.joblib")

    # Define phase-specific features (input parameters)
    phase_features = {
        "phase_1": ['FF', 'APUF_Mean', 'GS_Mean', 'Distance'],
        "phase_2": ['FF', 'N1T_Mean', 'GS_Mean', 'SAT', 'Distance'],
        "phase_3": ['FF', 'N1T_Mean', 'TAT', 'PS_Mean', 'Distance'],
        "phase_4": ['FF', 'N1T_Mean', 'ALTR_Mean', 'GS_Mean', 'Distance'],
        "phase_5": ['FF', 'TAS_Mean', 'MACH_Mean', 'TAT', 'Distance'],
        "phase_6": ['FF', 'GS_Mean', 'TAT', 'PT_Mean', 'Distance'],
        "phase_7": ['FF', 'GS_Mean', 'TRKM_Mean', 'LATP', 'Distance']
    }

    # Get the features for the specified phase
    features = phase_features.get(phase_name)
    if not features:
        print(f"Invalid phase name: {phase_name}")
        return None

    # Ensure that the input_data has the same order of features as expected by the model
    input_data = np.array(input_data).reshape(1, -1)

    # Standardize the input data using the loaded scaler
    input_data_scaled = scaler.transform(input_data)

    # Predict the fuel quantity change
    predicted_fq = model.predict(input_data_scaled)
    return predicted_fq[0]

# Specific input data based on the values provided
input_data= {
    "FF": 4465.221122,
    "N1T_Mean": 77.09976224,
    "APUF_Mean": 0,
    "ALT_Mean": 17731.18517,
    "ALTR_Mean": -13.87741631,
    "GS_Mean": 336.9670483,
    "TAS_Mean": 318.0284211,
    "MACH_Mean": 0.523525214,
    "DA_Mean": -3.118623179,
    "SAT": -17.87947902,
    "TAT": -2.337164074,
    "WD_Mean": -62.41109172,
    "WS_Mean": 57.20965571,
    "PS_Mean": 16.45111219,
    "PT_Mean": 659.2318445,
    "TOCW_Mean": 0,
    "TRK_Mean": 138.3261331,
    "TRKM_Mean": 136.4121042,
    "LATP": 41.39153364,
    "Distance": 723.2998876
}




# Prepare input data for each phase
phase_1_input = [input_data["FF"], input_data["APUF_Mean"], input_data["GS_Mean"], input_data["Distance"]]
phase_2_input = [input_data["FF"], input_data["N1T_Mean"], input_data["GS_Mean"], input_data["SAT"], input_data["Distance"]]
phase_3_input = [input_data["FF"], input_data["N1T_Mean"], input_data["TAT"], input_data["PS_Mean"], input_data["Distance"]]
phase_4_input = [input_data["FF"], input_data["N1T_Mean"], input_data["ALTR_Mean"], input_data["GS_Mean"], input_data["Distance"]]
phase_5_input = [input_data["FF"], input_data["TAS_Mean"], input_data["MACH_Mean"], input_data["TAT"], input_data["Distance"]]
phase_6_input = [input_data["FF"], input_data["GS_Mean"], input_data["TAT"], input_data["PT_Mean"], input_data["Distance"]]
phase_7_input = [input_data["FF"], input_data["GS_Mean"], input_data["TRKM_Mean"], input_data["LATP"], input_data["Distance"]]

# Example: Test for each phase
phases = {
    "phase_1": phase_1_input,
    "phase_2": phase_2_input,
    "phase_3": phase_3_input,
    "phase_4": phase_4_input,
    "phase_5": phase_5_input,
    "phase_6": phase_6_input,
    "phase_7": phase_7_input
}

# Predict for each phase and print the results
for phase_name, input_data in phases.items():
    predicted_fuel_quantity = predict_fuel_quantity(phase_name, input_data)
    print(f"Predicted fuel quantity change for {phase_name}: {predicted_fuel_quantity}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Predicted fuel quantity change for phase_1: 97.92
Predicted fuel quantity change for phase_2: 935.28
Predicted fuel quantity change for phase_3: 5641.12
Predicted fuel quantity change for phase_4: 4712.16
Predicted fuel quantity change for phase_5: 5592.16
Predicted fuel quantity change for phase_6: 3129.44
Predicted fuel quantity change for phase_7: -39.2


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
